# Data Transformation

In [2]:
#Dependencies
import pandas as pd
import requests
import json
from config import gkey
from sqlalchemy import create_engine

## School Data 

In [4]:
# read public school location data csv into df 
pb_loc_df = pd.read_csv('Resources/Data/School_Locations_(Public).csv')
pb_loc_df.head()

,CountyName,RecType,Region-2\nCounty-3\nDistrict-4,Type,School,FacilityName,Administrator,Mailing Address,Delivery Address,City,Zip,Telephone,GradeServed,State House District,StSen,FedCong,Cat,NCES ID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adams,Dist,010010010,26.0,0000,Payson CUSD 1,Dr. Donna Veile,406 W State St,NaN,Payson,62360 1041,217656-3323,P-12,94.0,47.0,18.0,2.0,1730990
2,Adams,Sch,010010010,26.0,0001,Seymour High School,Mrs. Dawn VanCamp,420 W Brainard St,NaN,Payson,62360 1033,217656-3355,7-12,94.0,47.0,18.0,4.0,173099003226
3,Adams,Sch,010010010,26.0,2002,Seymour Elementary School,Ms. Julie Phelan,404 W State St,NaN,Payson,62360 1041,217656-3323,P-6,94.0,47.0,18.0,4.0,173099003225
4,Adams,Dist,010010020,26.0,0000,Liberty CUSD 2,Mrs. Kelle Bunch,505 N Park St,NaN,Liberty,62347 1107,217645-3433,P-12,94.0,47.0,18.0,2.0,1722770


In [6]:
# read charter school location data csv into df
ch_loc_df = pd.read_csv('Resources/Data/School_Locations_(Charter).csv')
ch_loc_df.head()

,CountyName,RecType,Region-2\nCounty-3\nDistrict-4,Type,School,FacilityName,Administrator,Address,City,Zip,Telephone,GradeServed,State House District,StSen,FedCong,Cat,NCES ID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winnebago,District Authorized Charter Sch,41012050.0,25.0,203C,Jackson Charter School,Shavina Pierre,315 Summit St,Rockford,61107 4231,815316-0093,K-5,67.0,34.0,17.0,4.0,1.734510e+11
2,Winnebago,District Authorized Charter Sch,41012050.0,25.0,201C,Legacy Acad of Excellence Charter Sch,Barbara Forte,4029 Prairie Rd,Rockford,61102 4501,815961-1100,K-12,69.0,35.0,17.0,4.0,1.734510e+11
3,Winnebago,District Authorized Charter Sch,41012050.0,25.0,202C,Galapagos Rockford Charter Sch,Michael G Lane,2605 School St,Rockford,61101 5264,815708-7946,K-8,67.0,34.0,17.0,4.0,1.734510e+11
4,Cook,State Authorized Charter,70169010.0,25.0,0000,Southland College Prep Charter High School,Dr. Blondean Davis,4601 Sauk Trl,Richton Park,60471 1470,708748-0100,9-12,38.0,19.0,2.0,2.0,1.701400e+06


In [7]:
# concatenate CSVs
all_loc_df = pd.concat([pb_loc_df, ch_loc_df], axis=0, join="outer", sort="False")
all_loc_df.head()

,Address,Administrator,Cat,City,CountyName,Delivery Address,FacilityName,FedCong,GradeServed,Mailing Address,NCES ID,RecType,Region-2\nCounty-3\nDistrict-4,School,StSen,State House District,Telephone,Type,Zip
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Dr. Donna Veile,2.0,Payson,Adams,NaN,Payson CUSD 1,18.0,P-12,406 W State St,1730990,Dist,010010010,0000,47.0,94.0,217656-3323,26.0,62360 1041
2,NaN,Mrs. Dawn VanCamp,4.0,Payson,Adams,NaN,Seymour High School,18.0,7-12,420 W Brainard St,173099003226,Sch,010010010,0001,47.0,94.0,217656-3355,26.0,62360 1033
3,NaN,Ms. Julie Phelan,4.0,Payson,Adams,NaN,Seymour Elementary School,18.0,P-6,404 W State St,173099003225,Sch,010010010,2002,47.0,94.0,217656-3323,26.0,62360 1041
4,NaN,Mrs. Kelle Bunch,2.0,Liberty,Adams,NaN,Liberty CUSD 2,18.0,P-12,505 N Park St,1722770,Dist,010010020,0000,47.0,94.0,217645-3433,26.0,62347 1107


In [8]:
# drop irrelevent columns
locations_df = all_loc_df[['Address', 'Mailing Address', 'City', 'CountyName', 'FacilityName', 'RecType', 'Zip']]
locations_df.head()

,Address,Mailing Address,City,CountyName,FacilityName,RecType,Zip
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,406 W State St,Payson,Adams,Payson CUSD 1,Dist,62360 1041
2,NaN,420 W Brainard St,Payson,Adams,Seymour High School,Sch,62360 1033
3,NaN,404 W State St,Payson,Adams,Seymour Elementary School,Sch,62360 1041
4,NaN,505 N Park St,Liberty,Adams,Liberty CUSD 2,Dist,62347 1107


In [9]:
# fill NaN Address values with Mailing Address values
locations_df['Address'] = locations_df['Address'].fillna(locations_df['Mailing Address'])
loc2_df = locations_df.drop('Mailing Address', axis=1)
loc2_df.head()

C:\Users\Lorenza\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Address,City,CountyName,FacilityName,RecType,Zip
0,NaN,NaN,NaN,NaN,NaN,NaN
1,406 W State St,Payson,Adams,Payson CUSD 1,Dist,62360 1041
2,420 W Brainard St,Payson,Adams,Seymour High School,Sch,62360 1033
3,404 W State St,Payson,Adams,Seymour Elementary School,Sch,62360 1041
4,505 N Park St,Liberty,Adams,Liberty CUSD 2,Dist,62347 1107


In [10]:
# drop any missing zip or address values
loc2_df.dropna(subset=['Zip', 'Address'], axis=0, inplace=True)
loc2_df.head()

,Address,City,CountyName,FacilityName,RecType,Zip
1,406 W State St,Payson,Adams,Payson CUSD 1,Dist,62360 1041
2,420 W Brainard St,Payson,Adams,Seymour High School,Sch,62360 1033
3,404 W State St,Payson,Adams,Seymour Elementary School,Sch,62360 1041
4,505 N Park St,Liberty,Adams,Liberty CUSD 2,Dist,62347 1107
5,505 N Park St,Liberty,Adams,Liberty High School,Sch,62347 1107


In [11]:
# Split zip on space to retrieve 5 digit zip
loc2_df['Zip'] = loc2_df['Zip'].str.split(" ", expand=True)

# Rename FacilityName column
loc2_df.rename(columns={'FacilityName':'School Name'}, inplace=True)

# Drop Duplicate School Names
loc2_df.drop_duplicates(subset='School Name', inplace=True)


loc2_df.head()

,Address,City,CountyName,School Name,RecType,Zip
1,406 W State St,Payson,Adams,Payson CUSD 1,Dist,62360
2,420 W Brainard St,Payson,Adams,Seymour High School,Sch,62360
3,404 W State St,Payson,Adams,Seymour Elementary School,Sch,62360
4,505 N Park St,Liberty,Adams,Liberty CUSD 2,Dist,62347
5,505 N Park St,Liberty,Adams,Liberty High School,Sch,62347


In [12]:
# read in school performance CSVs

general_csv = 'Resources/Data/2019-Report-Card-Public-Data-Set (General).csv'
financial_csv = 'Resources/Data/2019-Report-Card-Public-Data-Set (Financial).csv'
sat_csv = 'Resources/Data/2019-Report-Card-Public-Data-Set (SAT).csv'

general_df = pd.read_csv(general_csv)
financials_df = pd.read_csv(financial_csv)
sat_df = pd.read_csv(sat_csv)

general_df.head()

,RCDTS,Type,School Name,District,City,County,District Type,District Size,School Type,Grades Served,...,State Performance Plan Indicator 7C2 - Met State Target?,State Performance Plan Indicator 8 - Met State Target?,State Performance Plan Indicator 9 - Met State Target?,State Performance Plan Indicator 10 - Met State Target?,State Performance Plan Indicator 11 - Met State Target?,State Performance Plan Indicator 12 - Met State Target?,State Performance Plan Indicator 13 - Met State Target?,State Performance Plan Indicator 14A - Met State Target?,State Performance Plan Indicator 14B - Met State Target?,State Performance Plan Indicator 14c - Met State Target?
0,010010010260000,District,NaN,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,NaN,NaN,...,Yes,NaN,Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN
1,010010010260001,School,Seymour High School,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,010010010262002,School,Seymour Elementary School,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,ELEMENTARY,PK K 1 2 3 4 5 6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010010020260000,District,NaN,Liberty CUSD 2,Liberty,Adams,UNIT,MEDIUM,NaN,NaN,...,No,NaN,Yes,Yes,Yes,NaN,Yes,NaN,NaN,NaN
4,010010020260001,School,Liberty High School,Liberty CUSD 2,Liberty,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Pull only necessary columns

gen_col_df = general_df[['RCDTS', 'School Name', 'School Type', 'County', '# Student Enrollment',\
                         '% Student Enrollment - Low Income', 'Student Attendance Rate']]

fin_col_df = financials_df[['RCDTS', 'School Name', '$ Site-level PEr-Pupil Expenditures - Subtotal',\
                            '$ District Centralized Per-Pupil Expenditure - Subtotal', '$ Total Per-Pupil Expenditures - Subtotal']]

sat_col_df = sat_df[['RCDTS', 'School Name', 'SAT Reading Average', 'SAT Math Average']]
sat_col_df.head()

,RCDTS,School Name,SAT Reading Average,SAT Math Average
0,010010010260000,NaN,470.9,454.1
1,010010010260001,Seymour High School,470.9,454.1
2,010010010262002,Seymour Elementary School,NaN,NaN
3,010010020260000,NaN,497.6,505.1
4,010010020260001,Liberty High School,497.6,505.1


In [14]:
# merge School Data CSVs on RCDTS #

merge1_df = gen_col_df.merge(fin_col_df, how="inner", on="RCDTS")
merge2_df = merge1_df.merge(sat_col_df, how="inner", on="RCDTS")

In [15]:
# Drop NaN school names
merge2_df.dropna(subset=['School Name'], axis=0, inplace=True)

# Drop Duplicate columns
school_data_df = merge2_df.drop(['School Name_y', 'School Name'], axis=1)

# Rename School Name_x column
school_data_df.rename(columns={'School Name_x': 'School Name'}, inplace=True)

# Pull only Cook County schools
cookco_df = school_data_df.loc[school_data_df['County'] == 'Cook']

# Drop Duplicate School Names
cookco_df.drop_duplicates(subset='School Name', inplace=True)

cookco_df.head()

C:\Users\Lorenza\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average
285,050160150041011,Plum Grove Jr High School,MIDDLE SCHL,Cook,814,13.4,96.2,8838.88,3224.66,12063.54,NaN,NaN
286,050160150041014,Carl Sandburg Jr High School,MIDDLE SCHL,Cook,524,45.0,95.2,10674.82,3617.67,14292.49,NaN,NaN
287,050160150041016,Winston Campus Jr High,MIDDLE SCHL,Cook,769,65.4,94.1,8817.97,3633.29,12451.26,NaN,NaN
288,050160150041018,Walter R Sundling Jr High Sch,MIDDLE SCHL,Cook,650,31.2,95.6,9983.04,3570.84,13553.88,NaN,NaN
289,050160150042001,Jane Addams Elem School,ELEMENTARY,Cook,579,66.5,94.7,8989.63,3098.94,12088.57,NaN,NaN


In [16]:
# Merge School Data with Location Data on School Name

final_df = cookco_df.merge(loc2_df, how="inner", on="School Name")
final_df.head()

,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip
0,050160150041011,Plum Grove Jr High School,MIDDLE SCHL,Cook,814,13.4,96.2,8838.88,3224.66,12063.54,NaN,NaN,2600 Plum Grove Rd,Rolling Meadows,Cook,Sch,60008
1,050160150041014,Carl Sandburg Jr High School,MIDDLE SCHL,Cook,524,45.0,95.2,10674.82,3617.67,14292.49,NaN,NaN,2600 Martin Ln,Rolling Meadows,Cook,Sch,60008
2,050160150041016,Winston Campus Jr High,MIDDLE SCHL,Cook,769,65.4,94.1,8817.97,3633.29,12451.26,NaN,NaN,120 N Babcock Dr,Palatine,Cook,Sch,60074
3,050160150041018,Walter R Sundling Jr High Sch,MIDDLE SCHL,Cook,650,31.2,95.6,9983.04,3570.84,13553.88,NaN,NaN,1100 N Smith St,Palatine,Cook,Sch,60067
4,050160150042001,Jane Addams Elem School,ELEMENTARY,Cook,579,66.5,94.7,8989.63,3098.94,12088.57,NaN,NaN,1020 E Sayles Dr,Palatine,Cook,Sch,60074


In [17]:
# Optional remove elementary/middle/charter schools

high_schools_df = final_df.loc[final_df['School Type'] == 'HIGH SCHOOL']
high_schools_df.head()


,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip
186,050162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201
187,050162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093
188,050162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093
189,050162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068
190,050162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068


In [18]:
# add columns for lat/lng

high_schools_df['Lat'] = ""
high_schools_df['Lng'] = ""

high_schools_df.head()

C:\Users\Lorenza\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lorenza\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip,Lat,Lng
186,050162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201,,
187,050162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093,,
188,050162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093,,
189,050162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068,,
190,050162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068,,


In [19]:
params ={"key": gkey}

for index, row in high_schools_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

    street = row['Address']
    city = row['City']

    # update address key value
    params['address'] = f"{street},{city},+IL"

    # make request
    schools_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    schools_lat_lng = schools_lat_lng.json()

    high_schools_df.loc[index, "Lat"] = schools_lat_lng["results"][0]["geometry"]["location"]["lat"]
    high_schools_df.loc[index, "Lng"] = schools_lat_lng["results"][0]["geometry"]["location"]["lng"]




C:\Users\Lorenza\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [20]:
# Visualize to confirm lat lng appear
high_schools_df.head()

,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip,Lat,Lng
186,050162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201,42.0475,-87.6998
187,050162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093,42.0944,-87.7193
188,050162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093,42.0949,-87.7668
189,050162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068,42.0374,-87.8576
190,050162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068,42.0014,-87.8523


In [21]:
high_schools_df.to_csv('Resources/Data/Cleaned/cleaned_schools_data.csv')

## Home values Data

In [22]:
#Open zillow home values files
homevalues_file = "Resources/Data/Zillow_homevalue_zipcodes.csv"
homevalues_df = pd.read_csv(homevalues_file)
homevalues_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,231064.0,...,1255626.0,1239473.0,1225833.0,1221555.0,1217736.0,1198491.0,1172006.0,1138563.0,1120424.0,1103971.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211597.0,...,495147.0,494290.0,493336.0,492329.0,491487.0,491249.0,492211.0,493494.0,494102.0,493717.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,236643.0,...,1128603.0,1114223.0,1105521.0,1108518.0,1127397.0,1146829.0,1160419.0,1161442.0,1171805.0,1178287.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,200691.0,...,336294.0,336438.0,336472.0,336463.0,336577.0,336340.0,336589.0,337122.0,338281.0,339317.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,287425.0,...,649878.0,648659.0,647684.0,646281.0,644917.0,644493.0,644516.0,644757.0,643758.0,642694.0


In [23]:
#Select everything that is Cook County and IL
homevalues_cook_df=homevalues_df.loc[(homevalues_df["State"] == "IL") & 
                            (homevalues_df["CountyName"] == "Cook County")]
homevalues_cook_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211597.0,...,495147.0,494290.0,493336.0,492329.0,491487.0,491249.0,492211.0,493494.0,494102.0,493717.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,287425.0,...,649878.0,648659.0,647684.0,646281.0,644917.0,644493.0,644516.0,644757.0,643758.0,642694.0
9,84640,9,60640,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,122919.0,...,311324.0,311253.0,311226.0,310688.0,310619.0,311165.0,312454.0,313755.0,314089.0,313859.0
17,84646,18,60647,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,159182.0,...,441969.0,441852.0,441601.0,441602.0,441464.0,442065.0,444343.0,447283.0,449930.0,451330.0
26,84620,28,60618,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,191027.0,...,449380.0,448361.0,447270.0,446141.0,445294.0,445585.0,447158.0,448927.0,449909.0,450240.0


In [24]:
# Check data
homevalues_cook_df.count()

RegionID      163
SizeRank      163
RegionName    163
RegionType    163
StateName     163
             ... 
2020-01-31    163
2020-02-29    163
2020-03-31    163
2020-04-30    163
2020-05-31    163
Length: 302, dtype: int64

In [25]:
# Create a filtered dataframe from specific columns (watchout for european date format!) to isolate 2019 values
homevalue_cols = ['RegionID', 'SizeRank', 'RegionName', 'State', 'City', 'Metro', 'CountyName', 
                     "2019-01-31","2019-02-28","2019-03-31","2019-04-30","2019-05-31",
                     "2019-06-30","2019-07-31","2019-08-31","2019-09-30","2019-10-31","2019-11-30","2019-12-31"]

homevalues_cook_2019_df = homevalues_cook_df[homevalue_cols]
homevalues_cook_2019_df.head()

,RegionID,SizeRank,RegionName,State,City,Metro,CountyName,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31
1,84654,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,500823.0,497702.0,495472.0,495042.0,495231.0,495084.0,495689.0,495147.0,494290.0,493336.0,492329.0,491487.0
4,84616,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,656398.0,652874.0,650747.0,651430.0,651464.0,650606.0,650524.0,649878.0,648659.0,647684.0,646281.0,644917.0
9,84640,9,60640,IL,Chicago,Chicago-Naperville-Elgin,Cook County,311913.0,309543.0,308026.0,308948.0,309995.0,310564.0,310999.0,311324.0,311253.0,311226.0,310688.0,310619.0
17,84646,18,60647,IL,Chicago,Chicago-Naperville-Elgin,Cook County,445419.0,442885.0,440848.0,440372.0,440812.0,440935.0,441744.0,441969.0,441852.0,441601.0,441602.0,441464.0
26,84620,28,60618,IL,Chicago,Chicago-Naperville-Elgin,Cook County,454906.0,452060.0,449913.0,449701.0,449934.0,449771.0,450086.0,449380.0,448361.0,447270.0,446141.0,445294.0


In [26]:
# Rename the column headers
homevalues_cook_2019_df = homevalues_cook_2019_df.rename(columns={"RegionName": "Zip"})
homevalues_cook_2019_df.head()

,RegionID,SizeRank,Zip,State,City,Metro,CountyName,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31
1,84654,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,500823.0,497702.0,495472.0,495042.0,495231.0,495084.0,495689.0,495147.0,494290.0,493336.0,492329.0,491487.0
4,84616,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,656398.0,652874.0,650747.0,651430.0,651464.0,650606.0,650524.0,649878.0,648659.0,647684.0,646281.0,644917.0
9,84640,9,60640,IL,Chicago,Chicago-Naperville-Elgin,Cook County,311913.0,309543.0,308026.0,308948.0,309995.0,310564.0,310999.0,311324.0,311253.0,311226.0,310688.0,310619.0
17,84646,18,60647,IL,Chicago,Chicago-Naperville-Elgin,Cook County,445419.0,442885.0,440848.0,440372.0,440812.0,440935.0,441744.0,441969.0,441852.0,441601.0,441602.0,441464.0
26,84620,28,60618,IL,Chicago,Chicago-Naperville-Elgin,Cook County,454906.0,452060.0,449913.0,449701.0,449934.0,449771.0,450086.0,449380.0,448361.0,447270.0,446141.0,445294.0


In [27]:
# Clean the data by dropping duplicates 
homevalues_cook_2019_df = homevalues_cook_2019_df.drop_duplicates("Zip")
homevalues_cook_2019_df["Zip"].count()

163

In [28]:
#Create a new column for average value for 2019
homevalues_cook_2019_df["AvgHomeValue"]=round(homevalues_cook_2019_df.iloc[:,6:19].mean(axis=1), 2)

homevalues_cook_2019_df.head()

,RegionID,SizeRank,Zip,State,City,Metro,CountyName,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,AvgHomeValue
1,84654,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,500823.0,497702.0,495472.0,495042.0,495231.0,495084.0,495689.0,495147.0,494290.0,493336.0,492329.0,491487.0,495136.00
4,84616,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,656398.0,652874.0,650747.0,651430.0,651464.0,650606.0,650524.0,649878.0,648659.0,647684.0,646281.0,644917.0,650121.83
9,84640,9,60640,IL,Chicago,Chicago-Naperville-Elgin,Cook County,311913.0,309543.0,308026.0,308948.0,309995.0,310564.0,310999.0,311324.0,311253.0,311226.0,310688.0,310619.0,310424.83
17,84646,18,60647,IL,Chicago,Chicago-Naperville-Elgin,Cook County,445419.0,442885.0,440848.0,440372.0,440812.0,440935.0,441744.0,441969.0,441852.0,441601.0,441602.0,441464.0,441791.92
26,84620,28,60618,IL,Chicago,Chicago-Naperville-Elgin,Cook County,454906.0,452060.0,449913.0,449701.0,449934.0,449771.0,450086.0,449380.0,448361.0,447270.0,446141.0,445294.0,449401.42


In [29]:
# export file
homevalues_cook_2019_df.to_csv("Resources/Data/Cleaned/clean_homevalue_2019.csv")

## Merging of csv

In [30]:
#Open Schools file
schools_data_df= pd.read_csv("Resources/Data/Cleaned/cleaned_schools_data.csv").drop(["Unnamed: 0"], axis=1)
schools_data_df.head()

,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip,Lat,Lng
0,50162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201,42.047544,-87.699769
1,50162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093,42.094380,-87.719263
2,50162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093,42.094887,-87.766762
3,50162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068,42.037380,-87.857584
4,50162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068,42.001374,-87.852285


In [31]:
#select only important columns from zillow dataset (ZIP and AverageValue)
homevalues_clean_df= homevalues_cook_2019_df[["Zip","AvgHomeValue"]]
homevalues_clean_df.head()

,Zip,AvgHomeValue
1,60657,495136.00
4,60614,650121.83
9,60640,310424.83
17,60647,441791.92
26,60618,449401.42


In [32]:
#merge df
merged_df= pd.merge(schools_data_df, homevalues_clean_df, on="Zip", left_index=True)
merged_df.head()

,RCDTS,School Name,School Type,County,# Student Enrollment,% Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip,Lat,Lng,AvgHomeValue
1226,50162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201,42.047544,-87.699769,511257.00
6656,50162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093,42.094380,-87.719263,1111715.17
6656,50162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093,42.094887,-87.766762,1111715.17
2120,50162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068,42.037380,-87.857584,421220.75
2120,50162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068,42.001374,-87.852285,421220.75


In [33]:
#Export as csv
merged_df.to_csv("Resources/Data/Cleaned/Schools_data_homevalue_merged.csv")

## Load data to SQL db

In [34]:
#Open zillow home values, school data and merged files
homevalues_cook_2019_df=pd.read_csv("Resources/Data/Cleaned/clean_homevalue_2019.csv").drop(["Unnamed: 0"], axis=1)
schools_data_df= pd.read_csv("Resources/Data/Cleaned/cleaned_schools_data.csv").drop(["Unnamed: 0"], axis=1)
merged_df= pd.read_csv("Resources/Data/Cleaned/Schools_data_homevalue_merged.csv").drop(["Unnamed: 0"], axis=1)

In [49]:
# Rename Columns with % to avoid running into errors with SQL
schools_data_df.rename(columns={"% Student Enrollment - Low Income": "Perc Student Enrollment - Low Income"}, inplace=True)
schools_data_df.head()

,RCDTS,School Name,School Type,County,# Student Enrollment,Perc Student Enrollment - Low Income,Student Attendance Rate,$ Site-level PEr-Pupil Expenditures - Subtotal,$ District Centralized Per-Pupil Expenditure - Subtotal,$ Total Per-Pupil Expenditures - Subtotal,SAT Reading Average,SAT Math Average,Address,City,CountyName,RecType,Zip,Lat,Lng
0,50162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201,42.047544,-87.699769
1,50162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093,42.094380,-87.719263
2,50162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093,42.094887,-87.766762
3,50162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068,42.037380,-87.857584
4,50162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068,42.001374,-87.852285


In [40]:
# Rename Columns with %, $, # and spaces to avoid running into errors with SQL and Flask app
merged_df.rename(columns={"School Name": "School_Name",
                          "School Type": "School_Type", 
                          "# Student Enrollment": "N_Student_Enrollment",
                          "% Student Enrollment Low Income": "Perc_Student_Enrollment_Low_Income",
                          "Student Attendance Rate": "Student_Attendance_Rate",
                          "$ Site-level PEr-Pupil Expenditures - Subtotal": "Site_level_Per_Pupil_Expenditures_Subtotal",
                          "$ District Centralized Per-Pupil Expenditure - Subtotal": "District_Centralized_Per_Pupil_Expenditure_Subtotal",
                          "$ Total Per-Pupil Expenditures - Subtotal": "Total_Per_Pupil_Expenditures_Subtotal",
                          "SAT Reading Average": "SAT_Reading_Average",
                          "SAT Math Average": "SAT_Math_Average",
                          "CountyName": "County_Name",
                          "RecType": "Rec_Type",
                          "AvgHomeValue": "Avg_Home_Value"}, inplace=True)
merged_df.head()

,RCDTS,School_Name,School_Type,County,N_Student_Enrollment,% Student Enrollment - Low Income,Student_Attendance_Rate,Site_level_Per_Pupil_Expenditures_Subtotal,District_Centralized_Per_Pupil_Expenditure_Subtotal,Total_Per_Pupil_Expenditures_Subtotal,SAT_Reading_Average,SAT_Math_Average,Address,City,County_Name,Rec_Type,Zip,Lat,Lng,Avg_Home_Value
0,50162020170001,Evanston Twp High School,HIGH SCHOOL,Cook,3514,36.5,91.5,20271.00,1949.00,22220.00,540.1,540.7,1600 Dodge Ave,Evanston,Cook,Sch,60201,42.047544,-87.699769,511257.00
1,50162030170001,New Trier Township H S Winnetka,HIGH SCHOOL,Cook,2963,4.2,94.9,22428.80,3076.44,25505.24,611.9,627.6,385 Winnetka Ave,Winnetka,Cook,Sch,60093,42.094380,-87.719263,1111715.17
2,50162030170002,New Trier Township H S Northfield,HIGH SCHOOL,Cook,1015,2.6,96.2,22281.03,3076.44,25357.47,NaN,NaN,7 Happ Rd,Northfield,Cook,Sch,60093,42.094887,-87.766762,1111715.17
3,50162070170001,Maine East High School,HIGH SCHOOL,Cook,1852,41.6,91.9,13382.03,5994.55,19376.58,491.2,497.3,2601 Dempster St,Park Ridge,Cook,Sch,60068,42.037380,-87.857584,421220.75
4,50162070170003,Maine South High School,HIGH SCHOOL,Cook,2391,6.4,93.8,12801.70,5919.13,18720.83,559.6,577.8,1111 S Dee Rd,Park Ridge,Cook,Sch,60068,42.001374,-87.852285,421220.75


In [44]:
#Create db connection
connection_string = "postgres:postgres@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{connection_string}')

In [45]:
# Confirm tables
engine.table_names()

['schools', 'homevalues', 'schools_homevalues']

In [46]:
#Load dataframes into databases
homevalues_cook_2019_df.to_sql(name='homevalues', con=engine, if_exists='append', index=False)

In [47]:
schools_data_df.to_sql(name='schools', con=engine, if_exists='append', index=False)

In [48]:
merged_df.to_sql(name='schools_homevalues', con=engine, if_exists='append', index=False)